
# 自作のデータセットを作成する

このチュートリアルは、すでに :doc:`ノード分類のためのGNNのトレーニングの基本 <1_introduction>` と :doc:`DGLグラフの作成、ロード、保存 <2_dglgraph>` についてて理解していることを想定している。

このチュートリアルでは、以下について学ぶ。

- 自作のグラフデータセットを作成し、ノード分類、リンク予測、グラフ分類に使用する

(Time estimate: 15 minutes)

## ``DGLDataset`` オブジェクトのの概要

カスタムグラフデータセットは、``dgl.data.DGLDataset`` クラスを継承し、以下のメソッドを実装する必要がある。

- ``__getitem__(self, i)``: i番目のデータを取得する。データは通常、1つのDGLグラフと、場合によってはラベルを含む。
- ``__len__(self)``: データセットのサイズ。
- ``process(self)``: ディスクから生データをロードし、前処理を行う。

## Creating a Dataset for Node Classification or Link Prediction from CSV

A node classification dataset often consists of a single graph, as well
as its node and edge features.

This tutorial takes a small dataset based on [Zachary’s Karate Club
network](https://en.wikipedia.org/wiki/Zachary%27s_karate_club)_. It
contains

* A ``members.csv`` file containing the attributes of all
  members, as well as their attributes.

* An ``interactions.csv`` file
  containing the pair-wise interactions between two club members.


## CSVファイルからノード分類のためのグラフデータセットを作成する

ノード分類データセットは、通常、1つのグラフとそのノードとエッジの特徴を含む。

このチュートリアルでは、[Zachary's Karate Club network](https://en.wikipedia.org/wiki/Zachary%27s_karate_club) に基づいて小規模データセットを使用する。これには以下が含まれる。

* ``members.csv`` ファイルには、すべてのメンバーの属性と、その属性が含まれている。

* ``interactions.csv`` ファイルには、2人のクラブメンバー間の相互作用が含まれている。

In [6]:
# データセットの読み込み
import urllib.request

import pandas as pd

urllib.request.urlretrieve(
    "https://data.dgl.ai/tutorial/dataset/members.csv", "./members.csv"
)
urllib.request.urlretrieve(
    "https://data.dgl.ai/tutorial/dataset/interactions.csv",
    "./interactions.csv",
)

members = pd.read_csv("./members.csv")
print(members.head())

interactions = pd.read_csv("./interactions.csv")
print(interactions.head())

   Id    Club  Age
0   0  Mr. Hi   44
1   1  Mr. Hi   37
2   2  Mr. Hi   37
3   3  Mr. Hi   40
4   4  Mr. Hi   30
   Src  Dst    Weight
0    0    1  0.043591
1    0    2  0.282119
2    0    3  0.370293
3    0    4  0.730570
4    0    5  0.821187


このチュートリアルでは、メンバーをノード、相互作用をエッジとして扱う。ノードの数値特徴として年齢、ノードのラベルとして所属クラブ、エッジの数値特徴としてエッジの重みを取る。

<div class="alert alert-info"><h4>Note</h4><p>元のZachary's Karate Club ネットワークはメンバーの年齢を保持していない。このチュートリアルで使用する年齢は、データの前処理でノード特徴量を加える方法を示すために人工的に生成されたものである。</p></div>

<div class="alert alert-info"><h4>Note</h4><p>実用的には、年齢の数値をそのまま特徴量とするのは機械学習においてはうまくいかないかもしれない; そのようなときにはビニングや標準化などが有効である。このチュートリアルでは簡単のため、値をそのまま使う。</p></div>

In [7]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import torch
from dgl.data import DGLDataset


class KarateClubDataset(DGLDataset): # KarateClubDatasetクラスの実装
    def __init__(self):
        super().__init__(name="karate_club")

    def process(self):
        nodes_data = pd.read_csv("./members.csv")
        edges_data = pd.read_csv("./interactions.csv")
        node_features = torch.from_numpy(nodes_data["Age"].to_numpy()) # 年齢を特徴量として使う
        node_labels = torch.from_numpy(
            nodes_data["Club"].astype("category").cat.codes.to_numpy()
        )
        edge_features = torch.from_numpy(edges_data["Weight"].to_numpy())
        edges_src = torch.from_numpy(edges_data["Src"].to_numpy())
        edges_dst = torch.from_numpy(edges_data["Dst"].to_numpy())

        self.graph = dgl.graph(
            (edges_src, edges_dst), num_nodes=nodes_data.shape[0]
        )
        self.graph.ndata["feat"] = node_features
        self.graph.ndata["label"] = node_labels
        self.graph.edata["weight"] = edge_features

        # ノード分類のデータセットの場合、ノードがトレーニング、バリデーション、テストセットに属しているかどうかを示すマスクを割り当てる必要がある
        n_nodes = nodes_data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train : n_train + n_val] = True
        test_mask[n_train + n_val :] = True
        self.graph.ndata["train_mask"] = train_mask
        self.graph.ndata["val_mask"] = val_mask
        self.graph.ndata["test_mask"] = test_mask

    def __getitem__(self, i):
        return self.graph # i番目といいつつ使うのは1グラフのみ

    def __len__(self):
        return 1


dataset = KarateClubDataset()
graph = dataset[0]

print(graph)

Graph(num_nodes=34, num_edges=156,
      ndata_schemes={'feat': Scheme(shape=(), dtype=torch.int64), 'label': Scheme(shape=(), dtype=torch.int8), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})


リンク予測データセットは1つのグラフのみを含むため、リンク予測データセットの作成はノード分類データセットと同じ前処理プロセスになる。

## CSV からグラフ分類のためのデータセットを作成する

グラフ分類データセットを作成するには、``__getitem__`` を実装して、グラフとグラフレベルのラベルの両方を返す必要がある。

このチュートリアルでは、以下の合成CSVデータを使用して、グラフ分類データセットを作成する方法を示す。

-  ``graph_edges.csv``: 3つのカラムを持つ:

   -  ``graph_id``: グラフのID
   -  ``src``: ソースノード
   -  ``dst``: ターゲットノード

-  ``graph_properties.csv``: 3つのカラムを持つ:

   -  ``graph_id``: グラフのID
   -  ``label``: グラフのラベル
   -  ``num_nodes``: グラフにおけるノードの数

In [8]:
urllib.request.urlretrieve(
    "https://data.dgl.ai/tutorial/dataset/graph_edges.csv", "./graph_edges.csv"
)
urllib.request.urlretrieve(
    "https://data.dgl.ai/tutorial/dataset/graph_properties.csv",
    "./graph_properties.csv",
)
edges = pd.read_csv("./graph_edges.csv")
properties = pd.read_csv("./graph_properties.csv")

edges.head()

properties.head()


class SyntheticDataset(DGLDataset): # 合成データセットの作成
    def __init__(self):
        super().__init__(name="synthetic")

    def process(self):
        edges = pd.read_csv("./graph_edges.csv")
        properties = pd.read_csv("./graph_properties.csv")
        self.graphs = []
        self.labels = []

        # エッジテーブルから各グラフIDに対してグラフを作成する
        # まず、プロパティテーブルをグラフIDをキーとする2つの辞書に変換する
        # ラベルとノード数が値となる
        label_dict = {}
        num_nodes_dict = {}
        for _, row in properties.iterrows():
            label_dict[row["graph_id"]] = row["label"]
            num_nodes_dict[row["graph_id"]] = row["num_nodes"]

        # エッジについては、まずグラフIDでテーブルをグループ化する
        edges_group = edges.groupby("graph_id")

        # 各グラフIDについて...
        for graph_id in edges_group.groups:
            # エッジとノード数とラベルを取得する。
            edges_of_id = edges_group.get_group(graph_id)
            src = edges_of_id["src"].to_numpy()
            dst = edges_of_id["dst"].to_numpy()
            num_nodes = num_nodes_dict[graph_id]
            label = label_dict[graph_id]

            # グラフを構築し、グラフリストとラベルリストに追加する
            g = dgl.graph((src, dst), num_nodes=num_nodes)
            self.graphs.append(g)
            self.labels.append(label)
            
        # ラベルリストを保存するためにテンソルに変換する
        self.labels = torch.LongTensor(self.labels)

    def __getitem__(self, i):
        return self.graphs[i], self.labels[i] # グラフ分類のときとは異なり、i番目のグラフとラベルを返す

    def __len__(self):
        return len(self.graphs)


dataset = SyntheticDataset()
graph, label = dataset[0]
print(graph, label)

Graph(num_nodes=15, num_edges=45,
      ndata_schemes={}
      edata_schemes={}) tensor(0)


## CSVから:class:`~dgl.data.CSVDataset`を使ってデータセットを作成する


これまでの例では、CSVファイルからデータセットを作成する手順を説明した。  
DGLは、CSVファイルからデータを読み込み、解析するためのユーティリティクラス:class:`~dgl.data.CSVDataset`を提供している。  
詳細は `guide-data-pipeline-loadcsv` を参照。

In [ ]:
# Thumbnail credits: (Un)common Use Cases for Graph Databases, Michal Bachman
# sphinx_gallery_thumbnail_path = '_static/blitz_6_load_data.png'